In [ ]:
# fold 5개로 나누기

import os
import shutil
import numpy as np

# 이미지 및 라벨 경로 설정
image_dir = 'K-Fold/images'
label_dir = 'K-Fold/labels'
output_dir = 'K-Fold/folds'

# 출력 디렉토리 생성
os.makedirs(output_dir, exist_ok=True)

# 이미지 파일 목록 가져오기
image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]

# 파일을 섞어서 5개의 폴드로 나누기
np.random.shuffle(image_files)

# 폴드 수
num_folds = 5
fold_size = len(image_files) // num_folds

for fold in range(num_folds):
    fold_dir = os.path.join(output_dir, f'fold_{fold}')
    os.makedirs(os.path.join(fold_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(fold_dir, 'labels'), exist_ok=True)

    # 현재 폴드에 해당하는 이미지와 라벨링 파일 복사
    for i in range(fold * fold_size, (fold + 1) * fold_size):
        if i < len(image_files):
            image_file = image_files[i]
            label_file = image_file.replace('.jpg', '.txt')  # 확장자 변경
            shutil.copy(os.path.join(image_dir, image_file), os.path.join(fold_dir, 'images'))
            shutil.copy(os.path.join(label_dir, label_file), os.path.join(fold_dir, 'labels'))

# 나머지 파일 처리 (있을 경우)
for i in range(num_folds * fold_size, len(image_files)):
    image_file = image_files[i]
    label_file = image_file.replace('.jpg', '.txt')
    fold_dir = os.path.join(output_dir, f'fold_{num_folds - 1}')  # 마지막 폴드에 추가
    shutil.copy(os.path.join(image_dir, image_file), os.path.join(fold_dir, 'images'))
    shutil.copy(os.path.join(label_dir, label_file), os.path.join(fold_dir, 'labels'))

print("데이터가 5개의 폴드로 나누어졌습니다.")


In [ ]:
#fold 5개로 나눈거 4개씩 합쳐서 train / val 나누기

import os
import shutil

# 원본 K-Fold 디렉토리 설정
original_folds_dir = 'K-Fold/folds'
output_dir = 'K-Fold/combined_folds'

# 출력 디렉토리 생성
os.makedirs(output_dir, exist_ok=True)

# 폴드 수 설정
num_folds = 5

# 각 묶음 생성
for bundle in range(num_folds):
    bundle_dir = os.path.join(output_dir, f'bundle_{bundle}')
    os.makedirs(os.path.join(bundle_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(bundle_dir, 'labels'), exist_ok=True)

    # 현재 묶음에 포함될 폴드 인덱스 계산
    for fold in range(4):  # 4개의 폴드를 묶음
        current_fold = (bundle + fold) % num_folds
        fold_dir = os.path.join(original_folds_dir, f'fold_{current_fold}')

        # 이미지 파일 복사
        for image_file in os.listdir(os.path.join(fold_dir, 'images')):
            if image_file.endswith('.jpg'):
                shutil.copy(os.path.join(fold_dir, 'images', image_file), os.path.join(bundle_dir, 'images'))

        # 라벨 파일 복사
        for label_file in os.listdir(os.path.join(fold_dir, 'labels')):
            if label_file.endswith('.txt'):
                shutil.copy(os.path.join(fold_dir, 'labels', label_file), os.path.join(bundle_dir, 'labels'))

print("데이터가 5개의 묶음으로 생성되었습니다.")


In [ ]:
#yaml파일 생성
import yaml

# yaml 파일 데이터 정의
data = {
    'train': 'train/images',
    'val': 'valid/images',
    'nc': 3,  # 클래스 수
    'names': ['Battery', 'Damaged','Pollution']  # 클래스 이름 리스트
}

# yaml 파일 생성
with open('K-Fold/fold_4/data.yaml', 'w') as file:
    yaml.dump(data, file, default_flow_style=False)

print("data.yaml 파일이 생성되었습니다.")

In [ ]:
import os
from ultralytics import YOLO
import wandb

# 폴드 수와 경로 설정
num_folds = 5
#base_fold_path = 'K-Fold/folds'  # 폴드 디렉토리 경로
data_config_template = 'K-Fold/fold_{}/data.yaml'  # 각 폴드에 맞는 data.yaml 경로

# 각 폴드에 대해 모델 학습
for fold in range(num_folds):
    # WandB 프로젝트 설정
    wandb.init(project='K-Fold', name='train_01')
    # YOLOv8 모델 로드
    model = YOLO('yolov8n.pt')  # YOLOv8의 다양한 버전 중 하나 선택

    # 각 폴드에 맞는 데이터셋 설정 파일 경로
    data_config = data_config_template.format(fold)

    print(f"폴드 {fold}에 대해 모델 학습 시작...")
    
    # 모델 학습
    model.train(data=data_config, epochs=200, imgsz=270, batch=16)

    print(f"폴드 {fold}에 대한 모델 학습이 완료되었습니다.")

print("모든 폴드에 대한 모델 학습이 완료되었습니다.")
